In [311]:
import tensorflow as tf
import glob
import numpy as np
import pandas as pd
from time import time
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
import random

from keras.models import Sequential
from keras.layers import Input, LSTM, Dropout, BatchNormalization, Dense, Conv1D, MaxPooling1D, Flatten, Concatenate
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

In [146]:
# jab_depth_files = [file for file in glob.glob('./numpy_data_arrays/jab/*depth.npz')]
# cross_depth_files = [file for file in glob.glob('./numpy_data_arrays/cross/*depth.npz')]
# left_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/left_hook/*depth.npz')]
# right_hook_depth_files = [file for file in glob.glob('./numpy_data_arrays/right_hook/*depth.npz')]
# random_depth_files = [file for file in glob.glob('./numpy_data_arrays/random/*depth.npz')]

# jab_color_files = [file for file in glob.glob('./numpy_data_arrays/jab/*color.npz')]
# cross_color_files = [file for file in glob.glob('./numpy_data_arrays/cross/*color.npz')]
# left_hook_color_files = [file for file in glob.glob('./numpy_data_arrays/left_hook/*color.npz')]
# right_hook_color_files = [file for file in glob.glob('./numpy_data_arrays/right_hook/*color.npz')]
# random_color_files = [file for file in glob.glob('./numpy_data_arrays/random/*color.npz')]

data_files = [file for file in glob.glob('./numpy_data_arrays/*/*.npz')]

row_names = []

for file in data_files:
    file_name = file.split("\\")[-1].split('.')[0]
    row_name = file_name[:-6]
    if not row_name in row_names: 
        row_names.append(row_name)


    

In [147]:
# lowest_num_of_frames = float('inf')
# for row_name in row_names:
#     number_of_frames = data_dict[row_name]['number_of_frames']
#     if number_of_frames < lowest_num_of_frames:
#         lowest_num_of_frames = number_of_frames

# lowest_num_of_frames = 6

# Extract 6 frames evenly spread out
def extract_frames(frames):
    extracted_frames = []
    for i in range(1, 7):
        frame_to_extract = (i * (frames.shape[0]) // 6 ) - 1
        extracted_frames.append(frames[frame_to_extract])
    return np.array(extracted_frames)

In [150]:
data_dict = {}

for file in data_files:
    file_name = file.split("\\")[-1].split('.')[0]
    row_name = file_name[:-6]
    depth_or_color = file_name[-5:]

    loaded_file = np.load(file)
    data = loaded_file['arr_0']
    print(data.shape)

    if row_name not in data_dict:
        data_dict[row_name] = {}

    if depth_or_color == 'color':
        data_dict[row_name]['full_color'] = data
        data_dict[row_name]['number_of_frames'] = data.shape[0]
        data_dict[row_name]['six_color_frames'] = extract_frames(data)
        if "jab" in row_name:
            data_dict[row_name]["label"] = "jab"
        elif "cross" in row_name:
            data_dict[row_name]["label"] = "cross"
        elif "left_hook" in row_name:
            data_dict[row_name]["label"] = "left_hook"
        elif "right_hook" in row_name:
            data_dict[row_name]["label"] = "right_hook"
        elif "random" in row_name:
            data_dict[row_name]["label"] = "random"
    elif depth_or_color == 'depth':
        data_dict[row_name]['full_depth'] = data
        data_dict[row_name]['six_depth_frames'] = extract_frames(data)

(21, 383, 383, 4)
(21, 383, 383)
(16, 383, 383, 4)
(16, 383, 383)
(28, 383, 383)
(28, 383, 383, 4)
(29, 383, 383)
(29, 383, 383, 4)
(21, 383, 383, 4)
(21, 383, 383)
(15, 383, 383)
(15, 383, 383, 4)
(18, 383, 383, 4)
(18, 383, 383)
(28, 383, 383)
(28, 383, 383, 4)
(30, 383, 383, 4)
(30, 383, 383)
(20, 383, 383, 4)
(20, 383, 383)
(38, 383, 383)
(38, 383, 383, 4)
(27, 383, 383, 4)
(27, 383, 383)
(30, 383, 383)
(30, 383, 383, 4)
(28, 383, 383)
(28, 383, 383, 4)
(30, 383, 383)
(30, 383, 383, 4)
(30, 383, 383)
(30, 383, 383, 4)
(19, 383, 383, 4)
(19, 383, 383)
(20, 383, 383)
(20, 383, 383, 4)
(6, 383, 383, 4)
(6, 383, 383)
(22, 383, 383)
(22, 383, 383, 4)
(10, 383, 383)
(10, 383, 383, 4)
(25, 383, 383, 4)
(25, 383, 383)
(17, 383, 383)
(17, 383, 383, 4)
(17, 383, 383, 4)
(17, 383, 383)
(21, 383, 383, 4)
(21, 383, 383)
(21, 383, 383)
(21, 383, 383, 4)
(26, 383, 383)
(26, 383, 383, 4)
(29, 383, 383)
(29, 383, 383, 4)
(24, 383, 383)
(24, 383, 383, 4)
(34, 383, 383)
(34, 383, 383, 4)
(37, 383, 38

In [162]:
# get max depth value
max_depth = float('-inf')
for row_name in row_names:
    six_depth_frames =  data_dict[row_name]['six_depth_frames']
    if six_depth_frames.max() > max_depth:
        max_depth = six_depth_frames.max()
    
print(max_depth)
# max_depth
# Max depth found = 3628
# bgr = data_dict['jab_1']['six_color_frames'][0][0][0][:-1]
# bgr2 = data_dict['jab_1']['six_color_frames'][0][0][0][:-1] / 255
# d = data_dict['jab_1']['six_depth_frames'][0][0][0] / 3628

# print(np.array([*bgr, d]))
# print(np.array([*bgr2, d]))

3628


In [264]:
for row_name in row_names:
    normalized_bgr = data_dict[row_name]['six_color_frames'][:][:][:][:] / 255
    normalized_d = data_dict[row_name]['six_depth_frames'][:][:][:] / 3628 #3628 found to be max
    normalized_bgr[..., 3] = normalized_d
    data_dict[row_name]['six_bgrd'] = np.array(normalized_bgr)

In [265]:
jab_dataset = [data_dict[row_name]["six_bgrd"] for row_name in row_names if "jab" in row_name]
cross_dataset = [data_dict[row_name]["six_bgrd"] for row_name in row_names if "cross" in row_name]
left_hook_dataset = [data_dict[row_name]["six_bgrd"] for row_name in row_names if "left_hook" in row_name]
right_hook_dataset = [data_dict[row_name]["six_bgrd"] for row_name in row_names if "right_hook" in row_name]
random_dataset = [data_dict[row_name]["six_bgrd"] for row_name in row_names if "random" in row_name]

In [305]:
random.shuffle(jab_dataset)
random.shuffle(cross_dataset)
random.shuffle(left_hook_dataset)
random.shuffle(right_hook_dataset)
random.shuffle(random_dataset)

X_train = jab_dataset[:19] + cross_dataset[:19] + left_hook_dataset[:19] + right_hook_dataset[:19] + random_dataset[:19]
y_train = ['jab'] * 19 + ['cross'] * 19 + ['left_hook'] * 19 + ['right_hook'] * 19 + ['random'] * 19
X_test = jab_dataset[19:] + cross_dataset[19:] + left_hook_dataset[19:] + right_hook_dataset[19:] + random_dataset[19:]
y_test = ['jab'] * 6 + ['cross'] * 6 + ['left_hook'] * 6 + ['right_hook'] * 6 + ['random'] * 6


In [306]:
X_train, y_train = shuffle(X_train, y_train)

In [312]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

classes = label_encoder.classes_
print("Unique Classes:", classes)
print("Encoded Training Labels:", y_train_encoded)
print("Encoded Testing Labels:", y_test_encoded)

Unique Classes: [1 2 3 4 5]
Encoded Training Labels: [0 4 0 1 2 0 4 4 2 3 3 3 0 4 1 1 3 2 4 0 2 3 4 1 0 1 2 4 3 3 1 0 3 1 2 2 2
 3 2 3 1 4 4 4 4 2 3 0 4 3 0 0 4 0 0 3 2 1 1 1 1 2 1 4 0 3 1 2 2 4 3 1 2 3
 3 4 1 1 0 3 0 1 2 0 2 2 2 0 0 4 4 1 0 4 3]
Encoded Testing Labels: [0 0 0 0 0 0 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3 4 4 4 4 4 4]


In [313]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', input_shape=(6, 383, 383, 4)),
    tf.keras.layers.MaxPooling3D((1, 2, 2)),
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((1, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

model.fit(X_train, y_train_encoded, epochs=10, batch_size=32)

model.evaluate(X_test, y_test_encoded)


Epoch 1/10


: 

: 

In [1]:
# Check if GPU is available
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

NameError: name 'tf' is not defined

In [ ]:
def iteration_2():
    time_at_start = int(time())
    conv_layers = [1, 2]
    conv_layer_sizes = [16, 32]
    dense_layers = [1, 2]
    dense_layer_sizes = [32, 64]

    EPOCHS = 50
    BATCH_SIZE = 32

    for dense_layer in dense_layers:
        for dense_layer_size in dense_layer_sizes:
                    for conv_layer in conv_layers:
                        for conv_layer_size in conv_layer_sizes:
                            name = f"{conv_layer}C{conv_layer_size}-{dense_layer}D{dense_layer_size}-{int(time())}"

                            model = Sequential()
                            model.add(Input(shape=(cnn_train_x.shape[1:])))

                            for layer in range(conv_layer - 1):
                                model.add(Conv1D(conv_layer_size, kernel_size=3, padding='valid', activation='relu'))
                                model.add(MaxPooling1D(1))
                    
                            model.add(Conv1D(conv_layer_size, kernel_size=3, padding='valid', activation='relu'))
                            model.add(MaxPooling1D(1))

                            model.add(Flatten())

                            for layer in range(dense_layer - 1):
                                model.add(Dense(dense_layer_size, activation='relu'))
                                model.add(Dropout(0.4))

                            model.add(Dense(2, activation='softmax'))

                            opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)

                            model.compile(loss='sparse_categorical_crossentropy',
                                            optimizer=opt,
                                            metrics=['accuracy'])

                            tensorboard = TensorBoard(log_dir=f'cnniteration2logs-{time_at_start}/{name}')

                            checkpoint_filepath = f"cnniteration2models-{time_at_start}/" + name + "-{epoch:02d}-{val_accuracy:.3f}.hd5"
                            checkpoint = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

                            early_stopping = EarlyStopping(monitor='val_accuracy', baseline=0.5, patience=12)

                            history = model.fit(
                                cnn_train_x, train_y,
                                batch_size=BATCH_SIZE,
                                epochs=EPOCHS,
                                validation_data=(cnn_validation_x, validation_y),
                                callbacks=[tensorboard, checkpoint, early_stopping]
                            )